<a href="https://colab.research.google.com/github/mrMichalR/Fake_news_detection_neural_network/blob/main/fake_news_detection_using_artificial_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake news detection using artificial neural networks


In [ ]:
!pip install pandas_profiling
!pip install torch
!pip install optuna
# !pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, sys
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.utils import resample
import copy
import re
import nltk
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
import torch.autograd as autograd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

<ipython-input-2-e743d551d629>:19: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/week2

Mounted at /content/drive
/content/drive/MyDrive/week2


## Datasets
Get datasets from Kaggle.

1. WELFake <br>
https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
2. Fake & Real <br>
https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=True.csv
3. Fake news net <br>
https://www.kaggle.com/datasets/algord/fake-news



In [ ]:
root = os.getcwd()
path = os.path.join(root, ".csv")
files_list=[]

for path, subdirs, files in os.walk(root):
  for name in files:
    temp_file = os.path.join(path, name)
    print(temp_file)
    files_list.append(temp_file)

/content/drive/MyDrive/week2/Fake.csv
/content/drive/MyDrive/week2/True.csv
/content/drive/MyDrive/week2/FakeNewsNet.csv
/content/drive/MyDrive/week2/WELFake_Dataset.csv
/content/drive/MyDrive/week2/output3.html
/content/drive/MyDrive/week2/output1.html
/content/drive/MyDrive/week2/output2.html
/content/drive/MyDrive/week2/ProfileReport1.html
/content/drive/MyDrive/week2/ProfileReport3.html
/content/drive/MyDrive/week2/ProfileReport2.html
/content/drive/MyDrive/week2/df2_downsampled_Fake_&_Real.csv
/content/drive/MyDrive/week2/df1_downsampled.csv
/content/drive/MyDrive/week2/df3_downsampled.csv
/content/drive/MyDrive/week2/df2_downsampled.csv


In [ ]:
df_fake = pd.read_csv(files_list[0])
df_true = pd.read_csv(files_list[1])
df_fake['fake'] = 1
df_true['fake'] = 0

df1 = pd.concat([df_fake, df_true], axis=0, ignore_index=False)
print(f'DataFrame of {files_list[0]} and {files_list[1]} file')
df1

DataFrame of /content/drive/MyDrive/week2/Fake.csv and /content/drive/MyDrive/week2/True.csv file


,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


**ATTENTION!** Due to high memory requirements taken:
- df1: 10000 samples
- df2: 15000 samples
- df3: 10000 samples

In [ ]:
df1 = df1.drop(['subject', 'date','text'], axis=1)
df1 = df1.sample(1000)
df1

,title,fake
5679,Treasury secretary to report on regulatory cha...,0
13948,Irish finance minister says prospect of electi...,0
16213,Egypt announces reshuffle in top security ranks,0
22137,MEMBERS: EP #5 – DRIVE BY WIRE: ‘Taxi to the U...,1
20837,Lesotho detains army major over killing of mil...,0
...,...,...
19309,HYPOCRISY ON STEROIDS: Check Out HATEFUL Trump...,1
4502,Homeland Security announces steps against H1B ...,0
13905,Algeria's ruling parties retain majority in lo...,0
3857,Seattle mayor drops re-election bid amid sex a...,0


In [ ]:
df2 = pd.read_csv(files_list[2])
df2.rename(columns = {'real':'fake'}, inplace = True)
df2 = df2.drop(['news_url', 'source_domain','tweet_num'], axis=1)
df2 = df2.sample(15000)
df2

,title,fake
9114,How many gold medals has Shaun White won at th...,1
16320,Serena Williams' Dad Pulled Out of Wedding One...,1
1336,Newly Single Mariah Carey and Ex Nick Cannon C...,1
11972,Sarah Michelle Gellar & Selma Blair Have a 'Cr...,1
17714,Medusa: MEG RYAN IS WASTING AWAY...,0
...,...,...
22154,Premio Lo Nuestro 2018: All the red carpet looks,1
21149,"Justin Bieber Warns Fans About False ""Glamorou...",1
8478,Candace Cameron Bure shared how she talks to h...,1
3377,"Southern Charm's Shep Talks Kathryn Hookups, D...",1


In [ ]:
df3 = pd.read_csv(files_list[3])
df3.rename(columns = {'label':'fake'}, inplace = True)
df3 = df3.drop(['Unnamed: 0','text'], axis=1)
df3 = df3.sample(1000)
df3

,title,fake
38461,Trump’s Latest Tease Pales Next to Bush and Go...,1
29056,John Kasich Will Create A Government Agency T...,1
27158,These 10 Seconds Of Sarah Palin’s INSANE Trum...,1
20869,"‘UFO’ Spotted Over Vienna, Frightened Onlooker...",1
50771,WHILE OBAMA VACATIONS AND HANDS OUT A BILLION ...,1
...,...,...
12469,Feds Seize $7.2 Million in Drugs at Border in Day,0
23338,"More Beer, Less Vodka as Russians Mull Ongoing...",1
4330,Still Not Thanking Native Americans,1
55822,"Trump raps Merkel over migrants, says U.S. cou...",0


All 3 in 1 list

In [ ]:
df_list = []
df_list.append(df1)
df_list.append(df2)
df_list.append(df3)

## Data analysis
Analyze the data and perform exploratory data analysis.

1. Check if the dataset is balanced - if not think about methods to acquire balance.
2. Use pandas-profiler
3. Visualize data using WordCloud.


In [ ]:
# check target value counts
target='fake'
for number, elem in zip(range(1,4), df_list):
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    534
0    466
Name: fake, dtype: int64
SUM:1000

Target value counts for df2:
1    11250
0     3750
Name: fake, dtype: int64
SUM:15000

Target value counts for df3:
1    506
0    494
Name: fake, dtype: int64
SUM:1000



In [ ]:
# check duplicates in dataset
for number, elem in zip(range(1,4), df_list):
  print(f'Duplicates for df{number}:\n{elem.duplicated(keep="last").sum()}\n')

Duplicates for df1:
3

Duplicates for df2:
632

Duplicates for df3:
8



In [ ]:
# drop duplicates in dataframes
for number, elem in zip(range(1,4), df_list):
  elem.drop_duplicates(keep="last", inplace=True)
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    531
0    466
Name: fake, dtype: int64
SUM:997

Target value counts for df2:
1    10827
0     3541
Name: fake, dtype: int64
SUM:14368

Target value counts for df3:
1    498
0    494
Name: fake, dtype: int64
SUM:992



In [ ]:
for number, elem in zip(range(1,4), df_list):
  print(f'Target imbalance for df{number}:\n{(elem[target].value_counts()[1]*100/elem[target].value_counts()[0]-100).round(2)} %\n')

Target imbalance for df1:
13.95 %

Target imbalance for df2:
205.76 %

Target imbalance for df3:
0.81 %



### Summary
- All df treated as unbalanced

For further investigation:
- Oversample minority class

or

- Undersample majority class


## Data preprocessing
Use preprocessing methods.

1. Think how to handle nulls and nan values

2. Propose other techniques to data preprocessing

In [ ]:
# check for nan values
for number, elem in zip(range(1,4), df_list):
  print(f'Missing values in df{number}:\n{elem.isna().sum()} %\n')

Missing values in df1:
title    0
fake     0
dtype: int64 %

Missing values in df2:
title    0
fake     0
dtype: int64 %

Missing values in df3:
title    1
fake     0
dtype: int64 %



In [ ]:
# drop nan values
for number, elem in zip(range(1,4), df_list):
  elem.dropna(inplace=True)
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    531
0    466
Name: fake, dtype: int64
SUM:997

Target value counts for df2:
1    10827
0     3541
Name: fake, dtype: int64
SUM:14368

Target value counts for df3:
1    497
0    494
Name: fake, dtype: int64
SUM:991



### Undersampling (majority class)

In [ ]:
df1_majority = df1[df1['fake'] == 0]
df1_minority = df1[(df1['fake'] == 1)]

df1_majority_downsampled = resample(df1_majority, replace = True, n_samples = len(df1_minority), random_state = 42)
df1_downsampled = pd.concat([df1_minority, df1_majority_downsampled])
df1_downsampled.to_csv('df1_downsampled.csv', index=False)

df1 = df1_downsampled
df_list[0] = df1_downsampled
print(f'Target value in df1 after udersampling:\n{df_list[0].fake.value_counts()}')


df2_majority = df2[df2['fake'] == 1]
df2_minority = df2[(df2['fake'] == 0)]

df2_majority_downsampled = resample(df2_majority, replace = True, n_samples = len(df2_minority), random_state = 42)
df2_downsampled = pd.concat([df2_minority, df2_majority_downsampled])
df2_downsampled.to_csv('df2_downsampled.csv', index=False)

df2 = df2_downsampled
df_list[1] = df2_downsampled
print(f'Target value in df2 after udersampling:\n{df_list[1].fake.value_counts()}')


df3_majority = df3[df3['fake'] == 0]
df3_minority = df3[(df3['fake'] == 1)]

df3_majority_downsampled = resample(df3_majority, replace = True, n_samples = len(df3_minority), random_state = 42)
df3_downsampled = pd.concat([df3_minority, df3_majority_downsampled])
df3_downsampled.to_csv('df3_downsampled.csv', index=False)

df3 = df3_downsampled
df_list[2] = df3_downsampled
print(f'Target value in df3 after udersampling:\n{df_list[2].fake.value_counts()}')

Target value in df1 after udersampling:
1    531
0    531
Name: fake, dtype: int64
Target value in df2 after udersampling:
0    3541
1    3541
Name: fake, dtype: int64
Target value in df3 after udersampling:
1    497
0    497
Name: fake, dtype: int64


## Data quality
Check the quality of your data

Source: https://www.geeksforgeeks.org/text-preprocessing-in-python-set-1/?ref=lbp  &  https://www.datacamp.com/tutorial/text-classification-python

1. Convert them to lower or upper case
2. Cut “stopwords”
3. Convert words to their stem or lemma
4. Plan how do you want to divide each sentence? There are a lot of tokenizers?
5. *remove all special characters
6. *remove digits
7. *remove punctuation
8. *remove whitespace

In [ ]:
def clean_data_keep_stopwords(dataframe):
  input_data = list(dataframe)
  corpus = []
  text = list(dataframe)
  for i in range(len(text)):
          # remove all special characters
          tokens = re.sub('[^a-zA-Z]', ' ', text[i])
          # remove digits
          tokens = re.sub(r'\d+', '', tokens)
          # to lower case
          tokens = tokens.lower()
          # tokenize
          tokens = word_tokenize(tokens)  #alternative tokens = tokens.split()
          # remove punctuation
          tokens = [word for word in tokens if word.isalpha()]
          # filter out short tokens (<=1 character)
          tokens = [word for word in tokens if len(word) > 1]
          # remove stopwords
          # tokens = [word for word in tokens if word not in stopwords.words('english')]
          # lemmatize
          tokens = [lemmatizer.lemmatize(word) for word in tokens]
          # tokens = [lemmatizer.lemmatize(word, pos ='v') for word in tokens] #lemmatize with provide context i.e. part-of-speech -> pos
          # remove whitespace
          tokens = ' '.join(tokens)

          corpus.append(tokens)

  # assign corpus to data column
  input_data = corpus
  return input_data

In [ ]:
def clean_data_without_lemma(dataframe):
   input_data = list(dataframe)
   corpus = []
   text = list(dataframe)
   for i in range(len(text)):
           # remove all special characters
           tokens = re.sub('[^a-zA-Z]', ' ', text[i])
           # remove digits
           tokens = re.sub(r'\d+', '', tokens)
           # to lower case
           tokens = tokens.lower()
           # tokenize
           tokens = word_tokenize(tokens)  #alternative tokens = tokens.split()
           # remove punctuation
           tokens = [word for word in tokens if word.isalpha()]
           # filter out short tokens (<=1 character)
           tokens = [word for word in tokens if len(word) > 1]
           # remove stopwords
           tokens = [word for word in tokens if word not in stopwords.words('english')]
           # lemmatize
           # tokens = [lemmatizer.lemmatize(word) for word in tokens]
           # tokens = [lemmatizer.lemmatize(word, pos ='v') for word in tokens] #lemmatize with provide context i.e. part-of-speech -> pos
           # remove whitespace
           tokens = ' '.join(tokens)

           corpus.append(tokens)
   # assign corpus to data column
   input_data = corpus
   return input_data

## Bag of words
Convert text to vector using classic algorithms

1. Use bag-of-words algorithm
2. Use Bag2Vec algorithm

Sources:

https://learn.microsoft.com/en-us/training/modules/intro-natural-language-processing-pytorch/3-bow-tfidf

WordCloud:

https://www.datacamp.com/tutorial/wordcloud-python


### Bag of words

In [ ]:
# Datasets cleaned but with kept stopwords
df_ws1 = pd.DataFrame()
df_ws2 = pd.DataFrame()
df_ws3 = pd.DataFrame()
df_ws1['title'] = clean_data_keep_stopwords(df1['title'])
df_ws2['title'] = clean_data_keep_stopwords(df2['title'])
df_ws3['title'] = clean_data_keep_stopwords(df3['title'])

# Datasets cleaned but without lemma
df_wol1 = pd.DataFrame()
df_wol2 = pd.DataFrame()
df_wol3 = pd.DataFrame()
df_wol1['title'] = clean_data_without_lemma(df1['title'])
df_wol2['title'] = clean_data_without_lemma(df2['title'])
df_wol3['title'] = clean_data_without_lemma(df3['title'])

In [ ]:
# Datasets vectors cleaned but with kept stopwords
vec_cv1 = CountVectorizer(ngram_range=(1,1)) #unigram = 1 word
vec_cv2 = CountVectorizer(ngram_range=(1,1))
vec_cv3 = CountVectorizer(ngram_range=(1,1))
df_ws1_vec = vec_cv1.fit_transform(df_ws1['title']).toarray()
df_ws2_vec = vec_cv2.fit_transform(df_ws2['title']).toarray()
df_ws3_vec = vec_cv3.fit_transform(df_ws3['title']).toarray()

# Datasets vectors cleaned but without lemma
vec_cv11 = CountVectorizer(ngram_range=(1,1)) #unigram = 1 word
vec_cv22 = CountVectorizer(ngram_range=(1,1))
vec_cv33 = CountVectorizer(ngram_range=(1,1))
df_wol1_vec = vec_cv11.fit_transform(df_wol1['title']).toarray()
df_wol2_vec = vec_cv22.fit_transform(df_wol2['title']).toarray()
df_wol3_vec = vec_cv33.fit_transform(df_wol3['title']).toarray()

As extras we can add code to keep words that occurs specific minimum times and reduce the size of voacb.

In [ ]:
# keep tokens/words with a min occurrence
min_occurence = 5
words_to_drop = []


bow_ws1 = pd.DataFrame(df_ws1_vec, columns = vec_cv1.get_feature_names_out())
bow_ws2 = pd.DataFrame(df_ws2_vec, columns = vec_cv2.get_feature_names_out())
bow_ws3 = pd.DataFrame(df_ws3_vec, columns = vec_cv3.get_feature_names_out())

bow_wol1 = pd.DataFrame(df_wol1_vec, columns = vec_cv11.get_feature_names_out())
bow_wol2 = pd.DataFrame(df_wol2_vec, columns = vec_cv22.get_feature_names_out())
bow_wol3 = pd.DataFrame(df_wol3_vec, columns = vec_cv33.get_feature_names_out())


def filter_words(bow_df):
  words_to_drop = []
  for col in bow_df.columns:
    if bow_df[col].sum() < min_occurence:
      words_to_drop.append(col)
  # print(f'Words to drop: {len(words_to_drop)}')
  bow_df_filter = bow_df.drop(words_to_drop, axis=1)
  # print(f'BoW vocab shape after reduction: {bow1_filter.shape}')
  return bow_df_filter


# ws => with stopwords and with lemma
bow_ws1_fil = filter_words(bow_ws1)
bow_ws1_fil_vocab_size = bow_ws1_fil.shape[1]
print(f'BoW 1 with kept stopwords shape: {bow_ws1_fil.shape}, length of vocab is: {bow_ws1_fil_vocab_size}')

bow_ws2_fil = filter_words(bow_ws2)
bow_ws2_fil_vocab_size = bow_ws2_fil.shape[1]
print(f'BoW 2 with kept stopwords shape: {bow_ws2_fil.shape}, length of vocab is: {bow_ws2_fil_vocab_size}')

bow_ws3_fil = filter_words(bow_ws3)
bow_ws3_fil_vocab_size = bow_ws3_fil.shape[1]
print(f'BoW 3 with kept stopwords shape: {bow_ws3_fil.shape}, length of vocab is: {bow_ws3_fil_vocab_size}')


# wol => with stopwords but without lemma
bow_wol1_fil = filter_words(bow_wol1)
bow_wol1_fil_vocab_size = bow_wol1_fil.shape[1]
print(f'BoW 1 without lemma shape: {bow_wol1_fil.shape}, length of vocab is: {bow_wol1_fil_vocab_size}')

bow_wol2_fil = filter_words(bow_wol2)
bow_wol2_fil_vocab_size = bow_wol2_fil.shape[1]
print(f'BoW 2 without lemma shape: {bow_wol2_fil.shape}, length of vocab is: {bow_wol2_fil_vocab_size}')

bow_wol3_fil = filter_words(bow_wol3)
bow_wol3_fil_vocab_size = bow_wol3_fil.shape[1]
print(f'BoW 3 without lemma shape: {bow_wol3_fil.shape}, length of vocab is: {bow_wol3_fil_vocab_size}')


def get_vocab_size(df):
  return df.shape[1]

BoW 1 with kept stopwords shape: (1062, 584), length of vocab is: 584
BoW 2 with kept stopwords shape: (7082, 2388), length of vocab is: 2388
BoW 3 with kept stopwords shape: (994, 486), length of vocab is: 486
BoW 1 without lemma shape: (1062, 475), length of vocab is: 475
BoW 2 without lemma shape: (7082, 2353), length of vocab is: 2353
BoW 3 without lemma shape: (994, 381), length of vocab is: 381


### TF-IDF

In [ ]:
# fit & transform
def tfidf_transform(df_features):
  tfidf = TfidfVectorizer()
  df = tfidf.fit_transform(df_features)
  vocab_size = len(tfidf.vocabulary_)
  print(f'Shape is: {df.shape}')
  print(f'Type: {type(df)}')
  print(f'VOCABULARY (lenght {vocab_size})\n{tfidf.vocabulary_}')
  # print(tfidf1.vocabulary_)
  # print(tfidf.idf_)
  return df, vocab_size

In [ ]:
# Datasets vectors cleaned but with kept stopwords
tfidf_ws1, tfidf_ws1_vocab_size = tfidf_transform(df_wol1['title'])
tfidf_ws2, tfidf_ws2_vocab_size = tfidf_transform(df_wol2['title'])
tfidf_ws3, tfidf_ws3_vocab_size = tfidf_transform(df_wol3['title'])

tfidf_ws1 = tfidf_ws1.toarray()
tfidf_ws2 = tfidf_ws2.toarray()
tfidf_ws3 = tfidf_ws3.toarray()


# Datasets vectors cleaned but without lemma
tfidf_wol1, tfidf_wol1_vocab_size = tfidf_transform(df1['title'])
tfidf_wol2, tfidf_wol2_vocab_size = tfidf_transform(df2['title'])
tfidf_wol3, tfidf_wol3_vocab_size = tfidf_transform(df3['title'])

tfidf_wol1 = tfidf_wol1.toarray()
tfidf_wol2 = tfidf_wol2.toarray()
tfidf_wol3 = tfidf_wol3.toarray()

Shape is: (1062, 3439)
Type: <class 'scipy.sparse._csr.csr_matrix'>
VOCABULARY (lenght 3439)
{'members': 1954, 'ep': 1027, 'drive': 940, 'wire': 3381, 'taxi': 3037, 'un': 3200, 'patrick': 2233, 'matt': 1927, 'lee': 1792, 'rick': 2592, 'scott': 2687, 'amped': 121, 'war': 3318, 'women': 3394, 'florida': 1193, 'democrats': 811, 'caught': 466, 'major': 1892, 'lie': 1822, 'dreamers': 937, 'video': 3269, 'actor': 37, 'vince': 3276, 'vaughn': 3253, 'destroys': 839, 'left': 1793, 'awesome': 212, 'statement': 2919, 'gun': 1349, 'rights': 2600, 'marco': 1910, 'roboto': 2620, 'rubio': 2635, 'short': 2784, 'circuits': 539, 'repeats': 2534, 'line': 1832, 'twice': 3186, 'seconds': 2706, 'hot': 1463, 'mic': 1971, 'republican': 2546, 'senator': 2725, 'calls': 424, 'congressman': 647, 'challenged': 485, 'duel': 952, 'fat': 1120, 'unattractive': 3203, 'moment': 2012, 'democratic': 810, 'friend': 1241, 'says': 2662, 'could': 685, 'beat': 255, 'sh': 2758, 'aggressor': 69, 'audio': 200, 'hello': 1413, 'uni

## Train, validation and test
Divide prepared dataset into three subsets.

Implement method to division dataset into training, validation and testing subsets.

Propose different divisions (in percentage):
1. 80/10/10 (tested in this notebook)
2. 70/15/15
3. 60/20/20

In [ ]:
# Create Feature and Label sets

# BoW with stopwords
X_bow_ws1 = bow_ws1_fil
y_bow_ws1 = df1['fake']

X_bow_ws2 = bow_ws2_fil
y_bow_ws2 = df2['fake']

X_bow_ws3 = bow_ws3_fil
y_bow_ws3 = df3['fake']

# BoW without lemma
X_bow_wol1 = bow_wol1_fil
y_bow_wol1 = df1['fake']

X_bow_wol2 = bow_wol2_fil
y_bow_wol2 = df2['fake']

X_bow_wol3 = bow_wol3_fil
y_bow_wol3 = df3['fake']


# TFIDF with stopwords
X_tfidf_ws1 = tfidf_ws1
y_tfidf_ws1 = df1['fake']

X_tfidf_ws2 = tfidf_ws2
y_tfidf_ws2 = df2['fake']

X_tfidf_ws3 = tfidf_ws3
y_tfidf_ws3 = df3['fake']

# TFIDF without lemma
X_tfidf_wol1 = tfidf_wol1
y_tfidf_wol1 = df1['fake']

X_tfidf_wol2 = tfidf_wol2
y_tfidf_wol2 = df2['fake']

X_tfidf_wol3 = tfidf_wol3
y_tfidf_wol3 = df3['fake']

In [ ]:
# train, val & test split (80% train, 10% val, 10% test)

# BOW
# split with stopwords
#df1
X_train1_ws, X_temp1_ws, y_train1_ws, y_temp1_ws = train_test_split(X_bow_ws1, y_bow_ws1, test_size=0.2, random_state=42)
X_val1_ws, X_test1_ws, y_val1_ws, y_test1_ws = train_test_split(X_temp1_ws, y_temp1_ws, test_size=0.5, random_state=42)

#df2
X_train2_ws, X_temp2_ws, y_train2_ws, y_temp2_ws = train_test_split(X_bow_ws2, y_bow_ws2, test_size=0.2, random_state=42)
X_val2_ws, X_test2_ws, y_val2_ws, y_test2_ws = train_test_split(X_temp2_ws, y_temp2_ws, test_size=0.5, random_state=42)

#df3
X_train3_ws, X_temp3_ws, y_train3_ws, y_temp3_ws = train_test_split(X_bow_ws3, y_bow_ws3, test_size=0.2, random_state=42)
X_val3_ws, X_test3_ws, y_val3_ws, y_test3_ws = train_test_split(X_temp3_ws, y_temp3_ws, test_size=0.5, random_state=42)

print('Dataset 1 BOW with stopwords:')
print('X train:', X_train1_ws.shape,'y train: ', y_train1_ws.shape)
print('X val: ', X_val1_ws.shape,'y val: ', y_val1_ws.shape)
print('X test:', X_test1_ws.shape,'y test: ', y_test1_ws.shape)
print('\n')

print('Dataset 2 BOW with stopwords:')
print('X train:', X_train2_ws.shape,'y train: ', y_train2_ws.shape)
print('X val: ', X_val2_ws.shape,'y val: ', y_val2_ws.shape)
print('X test:', X_test2_ws.shape,'y test: ', y_test2_ws.shape)
print('\n')


print('Dataset 3 BOW with stopwords:')
print('X train:', X_train3_ws.shape,'y train: ', y_train3_ws.shape)
print('X val: ', X_val3_ws.shape,'y val: ', y_val3_ws.shape)
print('X test:', X_test3_ws.shape,'y test: ', y_test3_ws.shape)
print('\n')

# BOW
# split without lemma
#df1
X_train1_wol, X_temp1_wol, y_train1_wol, y_temp1_wol = train_test_split(X_bow_wol1, y_bow_wol1, test_size=0.2, random_state=42)
X_val1_wol, X_test1_wol, y_val1_wol, y_test1_wol = train_test_split(X_temp1_wol, y_temp1_wol, test_size=0.5, random_state=42)

#df2
X_train2_wol, X_temp2_wol, y_train2_wol, y_temp2_wol = train_test_split(X_bow_wol2, y_bow_wol2, test_size=0.2, random_state=42)
X_val2_wol, X_test2_wol, y_val2_wol, y_test2_wol = train_test_split(X_temp2_wol, y_temp2_wol, test_size=0.5, random_state=42)

#df3
X_train3_wol, X_temp3_wol, y_train3_wol, y_temp3_wol = train_test_split(X_bow_wol3, y_bow_wol3, test_size=0.2, random_state=42)
X_val3_wol, X_test3_wol, y_val3_wol, y_test3_wol = train_test_split(X_temp3_wol, y_temp3_wol, test_size=0.5, random_state=42)

print('Dataset 1 BOW without lemma:')
print('X train:', X_train1_wol.shape,'y train: ', y_train1_wol.shape)
print('X val: ', X_val1_wol.shape,'y val: ', y_val1_wol.shape)
print('X test:', X_test1_wol.shape,'y test: ', y_test1_wol.shape)
print('\n')

print('Dataset 2 BOW without lemma:')
print('X train:', X_train2_wol.shape,'y train: ', y_train2_wol.shape)
print('X val: ', X_val2_wol.shape,'y val: ', y_val2_wol.shape)
print('X test:', X_test2_wol.shape,'y test: ', y_test2_wol.shape)
print('\n')


print('Dataset 3 BOW without lemma:')
print('X train:', X_train3_wol.shape,'y train: ', y_train3_wol.shape)
print('X val: ', X_val3_wol.shape,'y val: ', y_val3_wol.shape)
print('X test:', X_test3_wol.shape,'y test: ', y_test3_wol.shape)
print('\n')

Dataset 1 BOW with stopwords:
X train: (849, 584) y train:  (849,)
X val:  (106, 584) y val:  (106,)
X test: (107, 584) y test:  (107,)


Dataset 2 BOW with stopwords:
X train: (5665, 2388) y train:  (5665,)
X val:  (708, 2388) y val:  (708,)
X test: (709, 2388) y test:  (709,)


Dataset 3 BOW with stopwords:
X train: (795, 486) y train:  (795,)
X val:  (99, 486) y val:  (99,)
X test: (100, 486) y test:  (100,)


Dataset 1 BOW without lemma:
X train: (849, 475) y train:  (849,)
X val:  (106, 475) y val:  (106,)
X test: (107, 475) y test:  (107,)


Dataset 2 BOW without lemma:
X train: (5665, 2353) y train:  (5665,)
X val:  (708, 2353) y val:  (708,)
X test: (709, 2353) y test:  (709,)


Dataset 3 BOW without lemma:
X train: (795, 381) y train:  (795,)
X val:  (99, 381) y val:  (99,)
X test: (100, 381) y test:  (100,)




In [ ]:
#TFIDF split with stopwords
#df1
X_train1_tfidf_ws, X_temp1_tfidf_ws, y_train1_tfidf_ws, y_temp1_tfidf_ws = train_test_split(X_tfidf_ws1, y_tfidf_ws1, test_size=0.2, random_state=42)
X_val1_tfidf_ws, X_test1_tfidf_ws, y_val1_tfidf_ws, y_test1_tfidf_ws = train_test_split(X_temp1_tfidf_ws, y_temp1_tfidf_ws, test_size=0.5, random_state=42)

# #df2
X_train2_tfidf_ws, X_temp2_tfidf_ws, y_train2_tfidf_ws, y_temp2_tfidf_ws = train_test_split(X_tfidf_ws2, y_tfidf_ws2, test_size=0.2, random_state=42)
X_val2_tfidf_ws, X_test2_tfidf_ws, y_val2_tfidf_ws, y_test2_tfidf_ws = train_test_split(X_temp2_tfidf_ws, y_temp2_tfidf_ws, test_size=0.5, random_state=42)

# #df3
X_train3_tfidf_ws, X_temp3_tfidf_ws, y_train3_tfidf_ws, y_temp3_tfidf_ws = train_test_split(X_tfidf_ws3, y_tfidf_ws3, test_size=0.2, random_state=42)
X_val3_tfidf_ws, X_test3_tfidf_ws, y_val3_tfidf_ws, y_test3_tfidf_ws = train_test_split(X_temp3_tfidf_ws, y_temp3_tfidf_ws, test_size=0.5, random_state=42)

print('Dataset 1 TFIDF ws:')
print('X train:', X_train1_tfidf_ws.shape,'y train: ', y_train1_tfidf_ws.shape)
print('X val: ', X_val1_tfidf_ws.shape,'y val: ', y_val1_tfidf_ws.shape)
print('X test:', X_test1_tfidf_ws.shape,'y test: ', y_test1_tfidf_ws.shape)
print('\n')

print('Dataset 2 TFIDF ws:')
print('X train:', X_train2_tfidf_ws.shape,'y train: ', y_train2_tfidf_ws.shape)
print('X val: ', X_val2_tfidf_ws.shape,'y val: ', y_val2_tfidf_ws.shape)
print('X test:', X_test2_tfidf_ws.shape,'y test: ', y_test2_tfidf_ws.shape)
print('\n')

print('Dataset 3 TFIDF ws:')
print('X train:', X_train3_tfidf_ws.shape,'y train: ', y_train3_tfidf_ws.shape)
print('X val: ', X_val3_tfidf_ws.shape,'y val: ', y_val3_tfidf_ws.shape)
print('X test:', X_test3_tfidf_ws.shape,'y test: ', y_test3_tfidf_ws.shape)
print('\n')

# split without lemma
#df1
X_train1_tfidf_wol, X_temp1_tfidf_wol, y_train1_tfidf_wol, y_temp1_tfidf_wol = train_test_split(X_tfidf_wol1, y_tfidf_wol1, test_size=0.2, random_state=42)
X_val1_tfidf_wol, X_test1_tfidf_wol, y_val1_tfidf_wol, y_test1_tfidf_wol = train_test_split(X_temp1_tfidf_wol, y_temp1_tfidf_wol, test_size=0.5, random_state=42)

# #df2
X_train2_tfidf_wol, X_temp2_tfidf_wol, y_train2_tfidf_wol, y_temp2_tfidf_wol = train_test_split(X_tfidf_wol2, y_tfidf_wol2, test_size=0.2, random_state=42)
X_val2_tfidf_wol, X_test2_tfidf_wol, y_val2_tfidf_wol, y_test2_tfidf_wol = train_test_split(X_temp2_tfidf_wol, y_temp2_tfidf_wol, test_size=0.5, random_state=42)

# #df3
X_train3_tfidf_wol, X_temp3_tfidf_wol, y_train3_tfidf_wol, y_temp3_tfidf_wol = train_test_split(X_tfidf_wol3, y_tfidf_wol3, test_size=0.2, random_state=42)
X_val3_tfidf_wol, X_test3_tfidf_wol, y_val3_tfidf_wol, y_test3_tfidf_wol = train_test_split(X_temp3_tfidf_wol, y_temp3_tfidf_wol, test_size=0.5, random_state=42)

print('Dataset 1 TFIDF wol:')
print('X train:', X_train1_tfidf_wol.shape,'y train: ', y_train1_tfidf_wol.shape)
print('X val: ', X_val1_tfidf_wol.shape,'y val: ', y_val1_tfidf_wol.shape)
print('X test:', X_test1_tfidf_wol.shape,'y test: ', y_test1_tfidf_wol.shape)
print('\n')

print('Dataset 2 TFIDF wol:')
print('X train:', X_train2_tfidf_wol.shape,'y train: ', y_train2_tfidf_wol.shape)
print('X val: ', X_val2_tfidf_wol.shape,'y val: ', y_val2_tfidf_wol.shape)
print('X test:', X_test2_tfidf_wol.shape,'y test: ', y_test2_tfidf_wol.shape)
print('\n')

print('Dataset 3 TFIDF wol:')
print('X train:', X_train3_tfidf_wol.shape,'y train: ', y_train3_tfidf_wol.shape)
print('X val: ', X_val3_tfidf_wol.shape,'y val: ', y_val3_tfidf_wol.shape)
print('X test:', X_test3_tfidf_wol.shape,'y test: ', y_test3_tfidf_wol.shape)
print('\n')

Dataset 1 TFIDF ws:
X train: (849, 3439) y train:  (849,)
X val:  (106, 3439) y val:  (106,)
X test: (107, 3439) y test:  (107,)


Dataset 2 TFIDF ws:
X train: (5665, 9495) y train:  (5665,)
X val:  (708, 9495) y val:  (708,)
X test: (709, 9495) y test:  (709,)


Dataset 3 TFIDF ws:
X train: (795, 3405) y train:  (795,)
X val:  (99, 3405) y val:  (99,)
X test: (100, 3405) y test:  (100,)


Dataset 1 TFIDF wol:
X train: (849, 3609) y train:  (849,)
X val:  (106, 3609) y val:  (106,)
X test: (107, 3609) y test:  (107,)


Dataset 2 TFIDF wol:
X train: (5665, 9848) y train:  (5665,)
X val:  (708, 9848) y val:  (708,)
X test: (709, 9848) y test:  (709,)


Dataset 3 TFIDF wol:
X train: (795, 3592) y train:  (795,)
X val:  (99, 3592) y val:  (99,)
X test: (100, 3592) y test:  (100,)




## Custom deep neural network
Use PyTorch library.

1. Implement a more complex neural network.

2. Compare model performance between approaches using in all three projects.

3. Try to find similar news using cosine distance in vector space. Example:

Create tensors

In [ ]:
# BoW tensors for all dataset
# with stopwords
print('BOW with stopwords')
X1_bow_tens_ws = torch.tensor(X_bow_ws1.values, dtype=torch.float32)
y1_bow_tens_ws = torch.tensor(y_bow_ws1.values, dtype=torch.float32).reshape(-1,1)
print(X1_bow_tens_ws.shape,y1_bow_tens_ws.shape)

X2_bow_tens_ws = torch.tensor(X_bow_ws2.values, dtype=torch.float32)
y2_bow_tens_ws = torch.tensor(y_bow_ws2.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens_ws.shape,y2_bow_tens_ws.shape)

X3_bow_tens_ws = torch.tensor(X_bow_ws3.values, dtype=torch.float32)
y3_bow_tens_ws = torch.tensor(y_bow_ws3.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens_ws.shape,y3_bow_tens_ws.shape)

# without lemma
print('BOW without lemma')
X1_bow_tens_wol = torch.tensor(X_bow_wol1.values, dtype=torch.float32)
y1_bow_tens_wol = torch.tensor(y_bow_wol1.values, dtype=torch.float32).reshape(-1,1)
print(X1_bow_tens_wol.shape,y1_bow_tens_wol.shape)

X2_bow_tens_wol = torch.tensor(X_bow_wol2.values, dtype=torch.float32)
y2_bow_tens_wol = torch.tensor(y_bow_wol2.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens_wol.shape,y2_bow_tens_wol.shape)

X3_bow_tens_wol = torch.tensor(X_bow_wol3.values, dtype=torch.float32)
y3_bow_tens_wol = torch.tensor(y_bow_wol3.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens_wol.shape,y3_bow_tens_wol.shape)


# TFIDF tensors for all dataset
# with stopwords
print('TFIDF with stopwords')
X1_tfidf_tens_ws = torch.tensor(X_tfidf_ws1, dtype=torch.float32)
y1_tfidf_tens_ws = torch.tensor(y_tfidf_ws1.values, dtype=torch.float32).reshape(-1,1)
print(X1_tfidf_tens_ws.shape,y1_tfidf_tens_ws.shape)

X2_tfidf_tens_ws = torch.tensor(X_tfidf_ws2, dtype=torch.float32)
y2_tfidf_tens_ws = torch.tensor(y_tfidf_ws2.values, dtype=torch.float32).reshape(-1,1)
print(X2_tfidf_tens_ws.shape,y2_tfidf_tens_ws.shape)

X3_tfidf_tens_ws = torch.tensor(X_tfidf_ws3, dtype=torch.float32)
y3_tfidf_tens_ws = torch.tensor(y_tfidf_ws3.values, dtype=torch.float32).reshape(-1,1)
print(X3_tfidf_tens_ws.shape,y3_tfidf_tens_ws.shape)

# without lemma
print('TFIDF without lemma')
X1_tfidf_tens_wol = torch.tensor(X_tfidf_wol1, dtype=torch.float32)
y1_tfidf_tens_wol = torch.tensor(y_tfidf_wol1.values, dtype=torch.float32).reshape(-1,1)
print(X1_tfidf_tens_wol.shape,y1_tfidf_tens_wol.shape)

X2_tfidf_tens_wol = torch.tensor(X_tfidf_wol2, dtype=torch.float32)
y2_tfidf_tens_wol = torch.tensor(y_tfidf_wol2.values, dtype=torch.float32).reshape(-1,1)
print(X2_tfidf_tens_wol.shape,y2_tfidf_tens_wol.shape)

X3_tfidf_tens_wol = torch.tensor(X_tfidf_wol3, dtype=torch.float32)
y3_tfidf_tens_wol = torch.tensor(y_tfidf_wol3.values, dtype=torch.float32).reshape(-1,1)
print(X3_tfidf_tens_wol.shape,y3_tfidf_tens_wol.shape)

BOW with stopwords
torch.Size([1062, 584]) torch.Size([1062, 1])
torch.Size([7082, 2388]) torch.Size([7082, 1])
torch.Size([7082, 2388]) torch.Size([994, 1])
BOW without lemma
torch.Size([1062, 475]) torch.Size([1062, 1])
torch.Size([7082, 2353]) torch.Size([7082, 1])
torch.Size([7082, 2353]) torch.Size([994, 1])
TFIDF with stopwords
torch.Size([1062, 3439]) torch.Size([1062, 1])
torch.Size([7082, 9495]) torch.Size([7082, 1])
torch.Size([994, 3405]) torch.Size([994, 1])
TFIDF without lemma
torch.Size([1062, 3609]) torch.Size([1062, 1])
torch.Size([7082, 9848]) torch.Size([7082, 1])
torch.Size([994, 3592]) torch.Size([994, 1])


In [ ]:
# BOW with stopwords
# train
print('BOW with stopwords')
X1_tens_train_ws = torch.tensor(X_train1_ws.values, dtype=torch.float32)
y1_tens_train_ws = torch.tensor(y_train1_ws.values, dtype=torch.float32).reshape(-1,1)
print('BOW Train sets:\n',X1_tens_train_ws.shape,y1_tens_train_ws.shape)

X2_tens_train_ws = torch.tensor(X_train2_ws.values, dtype=torch.float32)
y2_tens_train_ws = torch.tensor(y_train2_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train_ws.shape,y2_tens_train_ws.shape)

X3_tens_train_ws = torch.tensor(X_train3_ws.values, dtype=torch.float32)
y3_tens_train_ws = torch.tensor(y_train3_ws.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train_ws.shape,y3_tens_train_ws.shape)

# val
X1_tens_val_ws = torch.tensor(X_val1_ws.values, dtype=torch.float32)
y1_tens_val_ws = torch.tensor(y_val1_ws.values, dtype=torch.float32).reshape(-1,1)
print('BOW Val sets:\n',X1_tens_val_ws.shape,y1_tens_val_ws.shape)

X2_tens_val_ws = torch.tensor(X_val2_ws.values, dtype=torch.float32)
y2_tens_val_ws = torch.tensor(y_val2_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val_ws.shape,y2_tens_val_ws.shape)

X3_tens_val_ws = torch.tensor(X_val3_ws.values, dtype=torch.float32)
y3_tens_val_ws = torch.tensor(y_val3_ws.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val_ws.shape,y3_tens_val_ws.shape)

# test
X1_tens_test_ws = torch.tensor(X_test1_ws.values, dtype=torch.float32)
y1_tens_test_ws = torch.tensor(y_test1_ws.values, dtype=torch.float32).reshape(-1,1)
print('BOW Test sets:\n',X1_tens_test_ws.shape,y1_tens_test_ws.shape)

X2_tens_test_ws = torch.tensor(X_test2_ws.values, dtype=torch.float32)
y2_tens_test_ws = torch.tensor(y_test2_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test_ws.shape,y2_tens_test_ws.shape)

X3_tens_test_ws = torch.tensor(X_test3_ws.values, dtype=torch.float32)
y3_tens_test_ws = torch.tensor(y_test3_ws.values, dtype=torch.float32).reshape(-1,1)
print(X1_tens_test_ws.shape,y1_tens_test_ws.shape)

# BOW without lemma
# train
print('BOW without lemma')
X1_tens_train_wol = torch.tensor(X_train1_wol.values, dtype=torch.float32)
y1_tens_train_wol = torch.tensor(y_train1_wol.values, dtype=torch.float32).reshape(-1,1)
print('BOW Train sets:\n',X1_tens_train_wol.shape,y1_tens_train_wol.shape)

X2_tens_train_wol = torch.tensor(X_train2_wol.values, dtype=torch.float32)
y2_tens_train_wol = torch.tensor(y_train2_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train_wol.shape,y2_tens_train_wol.shape)

X3_tens_train_wol = torch.tensor(X_train3_wol.values, dtype=torch.float32)
y3_tens_train_wol = torch.tensor(y_train3_wol.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train_wol.shape,y3_tens_train_wol.shape)

# val
X1_tens_val_wol = torch.tensor(X_val1_wol.values, dtype=torch.float32)
y1_tens_val_wol = torch.tensor(y_val1_wol.values, dtype=torch.float32).reshape(-1,1)
print('BOW Val sets:\n',X1_tens_val_wol.shape,y1_tens_val_wol.shape)

X2_tens_val_wol = torch.tensor(X_val2_wol.values, dtype=torch.float32)
y2_tens_val_wol = torch.tensor(y_val2_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val_wol.shape,y2_tens_val_wol.shape)

X3_tens_val_wol = torch.tensor(X_val3_wol.values, dtype=torch.float32)
y3_tens_val_wol = torch.tensor(y_val3_wol.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val_wol.shape,y3_tens_val_wol.shape)

# test
X1_tens_test_wol = torch.tensor(X_test1_wol.values, dtype=torch.float32)
y1_tens_test_wol = torch.tensor(y_test1_wol.values, dtype=torch.float32).reshape(-1,1)
print('BOW Test sets:\n',X1_tens_test_wol.shape,y1_tens_test_wol.shape)

X2_tens_test_wol = torch.tensor(X_test2_wol.values, dtype=torch.float32)
y2_tens_test_wol = torch.tensor(y_test2_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test_wol.shape,y2_tens_test_wol.shape)

X3_tens_test_wol = torch.tensor(X_test3_wol.values, dtype=torch.float32)
y3_tens_test_wol = torch.tensor(y_test3_wol.values, dtype=torch.float32).reshape(-1,1)
print(X1_tens_test_wol.shape,y1_tens_test_wol.shape)


BOW with stopwords
BOW Train sets:
 torch.Size([849, 584]) torch.Size([849, 1])
torch.Size([5665, 2388]) torch.Size([5665, 1])
torch.Size([795, 486]) torch.Size([795, 1])
BOW Val sets:
 torch.Size([106, 584]) torch.Size([106, 1])
torch.Size([708, 2388]) torch.Size([708, 1])
torch.Size([99, 486]) torch.Size([99, 1])
BOW Test sets:
 torch.Size([107, 584]) torch.Size([107, 1])
torch.Size([709, 2388]) torch.Size([709, 1])
torch.Size([107, 584]) torch.Size([107, 1])
BOW without lemma
BOW Train sets:
 torch.Size([849, 475]) torch.Size([849, 1])
torch.Size([5665, 2353]) torch.Size([5665, 1])
torch.Size([795, 381]) torch.Size([795, 1])
BOW Val sets:
 torch.Size([106, 475]) torch.Size([106, 1])
torch.Size([708, 2353]) torch.Size([708, 1])
torch.Size([99, 381]) torch.Size([99, 1])
BOW Test sets:
 torch.Size([107, 475]) torch.Size([107, 1])
torch.Size([709, 2353]) torch.Size([709, 1])
torch.Size([107, 475]) torch.Size([107, 1])


In [ ]:
# TFIDF with stopwords
print("TFIDF with stopwords")
#train
X1_tens_train_tfidf_ws = torch.tensor(X_train1_tfidf_ws, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_train_tfidf_ws = torch.tensor(y_train1_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF train:\n',X1_tens_train_tfidf_ws.shape,y1_tens_train_tfidf_ws.shape)

X2_tens_train_tfidf_ws = torch.tensor(X_train2_tfidf_ws, dtype=torch.float32)
y2_tens_train_tfidf_ws = torch.tensor(y_train2_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train_tfidf_ws.shape,y2_tens_train_tfidf_ws.shape)

X3_tens_train_tfidf_ws = torch.tensor(X_train3_tfidf_ws, dtype=torch.float32)
y3_tens_train_tfidf_ws = torch.tensor(y_train3_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train_tfidf_ws.shape,y3_tens_train_tfidf_ws.shape)

# val
X1_tens_val_tfidf_ws = torch.tensor(X_val1_tfidf_ws, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_val2).type(torch.float)
y1_tens_val_tfidf_ws = torch.tensor(y_val1_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF val:\n',X1_tens_val_tfidf_ws.shape,y1_tens_val_tfidf_ws.shape)

X2_tens_val_tfidf_ws = torch.tensor(X_val2_tfidf_ws, dtype=torch.float32)
y2_tens_val_tfidf_ws = torch.tensor(y_val2_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val_tfidf_ws.shape,y2_tens_val_tfidf_ws.shape)

X3_tens_val_tfidf_ws = torch.tensor(X_val3_tfidf_ws, dtype=torch.float32)
y3_tens_val_tfidf_ws = torch.tensor(y_val3_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val_tfidf_ws.shape,y3_tens_val_tfidf_ws.shape)

# test
X1_tens_test_tfidf_ws = torch.tensor(X_test1_tfidf_ws, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_test_tfidf_ws = torch.tensor(y_test1_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF test:\n',X1_tens_test_tfidf_ws.shape,y1_tens_test_tfidf_ws.shape)

X2_tens_test_tfidf_ws = torch.tensor(X_test2_tfidf_ws, dtype=torch.float32)
y2_tens_test_tfidf_ws = torch.tensor(y_test2_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test_tfidf_ws.shape,y2_tens_test_tfidf_ws.shape)

X3_tens_test_tfidf_ws = torch.tensor(X_test3_tfidf_ws, dtype=torch.float32)
y3_tens_test_tfidf_ws = torch.tensor(y_test3_tfidf_ws.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_test_tfidf_ws.shape,y3_tens_test_tfidf_ws.shape)



print("TFIDF without lemma")
#train
X1_tens_train_tfidf_wol = torch.tensor(X_train1_tfidf_wol, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_train_tfidf_wol = torch.tensor(y_train1_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF train:\n',X1_tens_train_tfidf_wol.shape,y1_tens_train_tfidf_wol.shape)

X2_tens_train_tfidf_wol = torch.tensor(X_train2_tfidf_wol, dtype=torch.float32)
y2_tens_train_tfidf_wol = torch.tensor(y_train2_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train_tfidf_wol.shape,y2_tens_train_tfidf_wol.shape)

X3_tens_train_tfidf_wol = torch.tensor(X_train3_tfidf_wol, dtype=torch.float32)
y3_tens_train_tfidf_wol = torch.tensor(y_train3_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train_tfidf_wol.shape,y3_tens_train_tfidf_wol.shape)

# val
X1_tens_val_tfidf_wol = torch.tensor(X_val1_tfidf_wol, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_val2).type(torch.float)
y1_tens_val_tfidf_wol = torch.tensor(y_val1_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF val:\n',X1_tens_val_tfidf_wol.shape,y1_tens_val_tfidf_wol.shape)

X2_tens_val_tfidf_wol = torch.tensor(X_val2_tfidf_wol, dtype=torch.float32)
y2_tens_val_tfidf_wol = torch.tensor(y_val2_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val_tfidf_wol.shape,y2_tens_val_tfidf_wol.shape)

X3_tens_val_tfidf_wol = torch.tensor(X_val3_tfidf_wol, dtype=torch.float32)
y3_tens_val_tfidf_wol = torch.tensor(y_val3_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val_tfidf_wol.shape,y3_tens_val_tfidf_wol.shape)

# test
X1_tens_test_tfidf_wol = torch.tensor(X_test1_tfidf_wol, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_test_tfidf_wol = torch.tensor(y_test1_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF test:\n',X1_tens_test_tfidf_wol.shape,y1_tens_test_tfidf_wol.shape)

X2_tens_test_tfidf_wol = torch.tensor(X_test2_tfidf_wol, dtype=torch.float32)
y2_tens_test_tfidf_wol = torch.tensor(y_test2_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test_tfidf_wol.shape,y2_tens_test_tfidf_wol.shape)

X3_tens_test_tfidf_wol = torch.tensor(X_test3_tfidf_wol, dtype=torch.float32)
y3_tens_test_tfidf_wol = torch.tensor(y_test3_tfidf_wol.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_test_tfidf_wol.shape,y3_tens_test_tfidf_wol.shape)

TFIDF with stopwords
TFIDF train:
 torch.Size([849, 3439]) torch.Size([849, 1])
torch.Size([5665, 9495]) torch.Size([5665, 1])
torch.Size([795, 3405]) torch.Size([795, 1])
TFIDF val:
 torch.Size([106, 3439]) torch.Size([106, 1])
torch.Size([708, 9495]) torch.Size([708, 1])
torch.Size([99, 3405]) torch.Size([99, 1])
TFIDF test:
 torch.Size([107, 3439]) torch.Size([107, 1])
torch.Size([709, 9495]) torch.Size([709, 1])
torch.Size([100, 3405]) torch.Size([100, 1])
TFIDF without lemma
TFIDF train:
 torch.Size([849, 3609]) torch.Size([849, 1])
torch.Size([5665, 9848]) torch.Size([5665, 1])
torch.Size([795, 3592]) torch.Size([795, 1])
TFIDF val:
 torch.Size([106, 3609]) torch.Size([106, 1])
torch.Size([708, 9848]) torch.Size([708, 1])
torch.Size([99, 3592]) torch.Size([99, 1])
TFIDF test:
 torch.Size([107, 3609]) torch.Size([107, 1])
torch.Size([709, 9848]) torch.Size([709, 1])
torch.Size([100, 3592]) torch.Size([100, 1])


In [ ]:
# build custom module/net Logistic regression
class Wide(nn.Module):
  def __init__(self, vocab_size, num_labels):
    super().__init__()
    self.hidden = nn.Linear(vocab_size, num_labels)
    self.relu = nn.ReLU()
    self.output = nn.Linear(num_labels, 1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    x = self.relu(self.hidden(x))
    x = self.sigmoid(self.output(x))
    return x

In [ ]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    n_epochs = 10   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())

    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

Evaluate model on each df test subset

In [ ]:
# create model, train, and get accuracy
model1 = Wide(get_vocab_size(X1_tens_train_ws),1)
acc1 = model_train(model1, X1_tens_train_ws, y1_tens_train_ws, X1_tens_val_ws, y1_tens_val_ws)
print("BOW df1 with stopwords\n",'Accuracy (Wide net): %.2f' % acc1)

# evaluate the model on the test set
model1.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred1 = model1(X1_tens_test_ws[i:i+1])
    print(f"{y_pred1[0].numpy()} " +
      f"(expected {y1_tens_test_ws[i].numpy()})")



# create model, train, and get accuracy
model2 = Wide(get_vocab_size(X2_tens_train_ws),1)
acc2 = model_train(model2, X2_tens_train_ws, y2_tens_train_ws, X2_tens_val_ws, y2_tens_val_ws)
print("BOW df2 with stopwords\n",'Accuracy (Wide net): %.2f' % acc2)

# evaluate the model on the test set
model2.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred2 = model2(X2_tens_test_ws[i:i+1])
    print(f"{y_pred2[0].numpy()} " +
      f"(expected {y2_tens_test_ws[i].numpy()})")



# create model, train, and get accuracy
model3 = Wide(get_vocab_size(X3_tens_train_ws),1)
acc3 = model_train(model3, X3_tens_train_ws, y3_tens_train_ws, X3_tens_val_ws, y3_tens_val_ws)
print("BOW df3 with stopwords\n",'Accuracy (Wide net): %.2f' % acc3)

# evaluate the model on the test set
model3.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred3 = model3(X3_tens_test_ws[i:i+1])
    print(f"{y_pred3[0].numpy()} " +
      f"(expected {y3_tens_test_ws[i].numpy()})")

BOW df1 with stopwords
 Accuracy (Wide net): 0.88
[0.38141575] (expected [0.])
[0.47684026] (expected [0.])
[0.52323556] (expected [1.])
[0.44941038] (expected [1.])
[0.6107471] (expected [1.])
BOW df2 with stopwords
 Accuracy (Wide net): 0.72
[0.31503987] (expected [0.])
[0.23358919] (expected [0.])
[0.37757683] (expected [1.])
[0.49660447] (expected [1.])
[0.6068001] (expected [0.])
BOW df3 with stopwords
 Accuracy (Wide net): 0.73
[0.49587804] (expected [0.])
[0.5023342] (expected [1.])
[0.5024635] (expected [0.])
[0.5199887] (expected [1.])
[0.49789768] (expected [0.])


In [ ]:
# create model, train, and get accuracy
model11 = Wide(get_vocab_size(X1_tens_train_wol),1)
acc11 = model_train(model11, X1_tens_train_wol, y1_tens_train_wol, X1_tens_val_wol, y1_tens_val_wol)
print("BOW df1 without lemma\n",'Accuracy (Wide net): %.2f' % acc11)

# evaluate the model on the test set
model11.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred1 = model11(X1_tens_test_wol[i:i+1])
    print(f"{y_pred1[0].numpy()} " +
      f"(expected {y1_tens_test_wol[i].numpy()})")



# create model, train, and get accuracy
model22 = Wide(get_vocab_size(X2_tens_train_wol),1)
acc22 = model_train(model22, X2_tens_train_wol, y2_tens_train_wol, X2_tens_val_wol, y2_tens_val_wol)
print("BOW df2 without lemma\n",'Accuracy (Wide net): %.2f' % acc22)

# evaluate the model on the test set
model22.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred2 = model22(X2_tens_test_wol[i:i+1])
    print(f"{y_pred2[0].numpy()} " +
      f"(expected {y2_tens_test_wol[i].numpy()})")



# create model, train, and get accuracy
model33 = Wide(get_vocab_size(X3_tens_train_wol),1)
acc33 = model_train(model33, X3_tens_train_wol, y3_tens_train_wol, X3_tens_val_wol, y3_tens_val_wol)
print("BOW df3 without lemma\n",'Accuracy (Wide net): %.2f' % acc33)

# evaluate the model on the test set
model33.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred3 = model33(X3_tens_test_wol[i:i+1])
    print(f"{y_pred3[0].numpy()} " +
      f"(expected {y3_tens_test_wol[i].numpy()})")

BOW df1 without lemma
 Accuracy (Wide net): 0.70
[0.40846437] (expected [0.])
[0.47155145] (expected [0.])
[0.45039526] (expected [1.])
[0.4819025] (expected [1.])
[0.5339627] (expected [1.])
BOW df2 without lemma
 Accuracy (Wide net): 0.74
[0.4013006] (expected [0.])
[0.23888208] (expected [0.])
[0.3665357] (expected [1.])
[0.4917606] (expected [1.])
[0.60846347] (expected [0.])
BOW df3 without lemma
 Accuracy (Wide net): 0.48
[0.352134] (expected [0.])
[0.35220978] (expected [1.])
[0.352134] (expected [0.])
[0.352134] (expected [1.])
[0.352134] (expected [0.])


TFIDF

In [ ]:
# create model, train, and get accuracy
model4 = Wide(get_vocab_size(X1_tens_train_tfidf_ws),1)
acc4 = model_train(model4, X1_tens_train_tfidf_ws, y1_tens_train_tfidf_ws, X1_tens_val_tfidf_ws, y1_tens_val_tfidf_ws)
print("TFIDF df1 with stopwords\n",'Accuracy (Wide net): %.2f' % acc4)

# evaluate the model on the test set
model4.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred4 = model4(X1_tens_test_tfidf_ws[i:i+1])
    print(f"{y_pred4[0].numpy()} " +
      f"(expected {y1_tens_test_tfidf_ws[i].numpy()})")



# create model, train, and get accuracy
model5 = Wide(get_vocab_size(X2_tens_train_tfidf_ws),1)
acc5 = model_train(model5, X2_tens_train_tfidf_ws, y2_tens_train_tfidf_ws, X2_tens_val_tfidf_ws, y2_tens_val_tfidf_ws)
print("TFIDF df2 with stopwords\n",'Accuracy (Wide net): %.2f' % acc5)

# evaluate the model on the test set
model5.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred5 = model5(X2_tens_test_tfidf_ws[i:i+1])
    print(f"{y_pred5[0].numpy()} " +
      f"(expected {y2_tens_test_tfidf_ws[i].numpy()})")



# create model, train, and get accuracy
model6 = Wide(get_vocab_size(X3_tens_train_tfidf_ws),1)
acc6 = model_train(model6, X3_tens_train_tfidf_ws, y3_tens_train_tfidf_ws, X3_tens_val_tfidf_ws, y3_tens_val_tfidf_ws)
print("TFIDF df3 with stopwords\n",'Accuracy (Wide net): %.2f' % acc6)

# evaluate the model on the test set
model6.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred6 = model6(X3_tens_test_tfidf_ws[i:i+1])
    print(f"{y_pred6[0].numpy()} " +
      f"(expected {y3_tens_test_tfidf_ws[i].numpy()})")

TFIDF df1 with stopwords
 Accuracy (Wide net): 0.58
[0.5893768] (expected [0.])
[0.5893768] (expected [0.])
[0.5893768] (expected [1.])
[0.58994013] (expected [1.])
[0.5893768] (expected [1.])
TFIDF df2 with stopwords
 Accuracy (Wide net): 0.70
[0.4961118] (expected [0.])
[0.4905563] (expected [0.])
[0.49931324] (expected [1.])
[0.5033025] (expected [1.])
[0.5141262] (expected [0.])
TFIDF df3 with stopwords
 Accuracy (Wide net): 0.48
[0.348803] (expected [0.])
[0.348803] (expected [1.])
[0.348803] (expected [0.])
[0.34870976] (expected [1.])
[0.34848398] (expected [0.])


In [ ]:
# create model, train, and get accuracy
model44 = Wide(get_vocab_size(X1_tens_train_tfidf_wol),1)
acc44 = model_train(model44, X1_tens_train_tfidf_wol, y1_tens_train_tfidf_wol, X1_tens_val_tfidf_wol, y1_tens_val_tfidf_wol)
print("TFIDF df1 without lemma\n",'Accuracy (Wide net): %.2f' % acc44)

# evaluate the model on the test set
model44.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred44 = model44(X1_tens_test_tfidf_wol[i:i+1])
    print(f"{y_pred44[0].numpy()} " +
      f"(expected {y1_tens_test_tfidf_wol[i].numpy()})")



# create model, train, and get accuracy
model55 = Wide(get_vocab_size(X2_tens_train_tfidf_wol),1)
acc55 = model_train(model55, X2_tens_train_tfidf_wol, y2_tens_train_tfidf_wol, X2_tens_val_tfidf_wol, y2_tens_val_tfidf_wol)
print("TFIDF df2 without lemma\n",'Accuracy (Wide net): %.2f' % acc55)

# evaluate the model on the test set
model55.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred55 = model55(X2_tens_test_tfidf_wol[i:i+1])
    print(f"{y_pred55[0].numpy()} " +
      f"(expected {y2_tens_test_tfidf_wol[i].numpy()})")



# create model, train, and get accuracy
model66 = Wide(get_vocab_size(X3_tens_train_tfidf_wol),1)
acc66 = model_train(model66, X3_tens_train_tfidf_wol, y3_tens_train_tfidf_wol, X3_tens_val_tfidf_wol, y3_tens_val_tfidf_wol)
print("TFIDF df3 without lemma\n",'Accuracy (Wide net): %.2f' % acc66)

# evaluate the model on the test set
model66.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred66 = model66(X3_tens_test_tfidf_wol[i:i+1])
    print(f"{y_pred66[0].numpy()} " +
      f"(expected {y3_tens_test_tfidf_wol[i].numpy()})")

TFIDF df1 without lemma
 Accuracy (Wide net): 0.58
[0.5643971] (expected [0.])
[0.5643971] (expected [0.])
[0.5643971] (expected [1.])
[0.5643971] (expected [1.])
[0.5643971] (expected [1.])
TFIDF df2 without lemma
 Accuracy (Wide net): 0.52
[0.55662096] (expected [0.])
[0.55662096] (expected [0.])
[0.55662096] (expected [1.])
[0.55662096] (expected [1.])
[0.55662096] (expected [0.])
TFIDF df3 without lemma
 Accuracy (Wide net): 0.52
[0.66630894] (expected [0.])
[0.66630894] (expected [1.])
[0.66630894] (expected [0.])
[0.66630894] (expected [1.])
[0.66630894] (expected [0.])


### Cross-validation

In [ ]:
# BOW 5-fold cross-validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)

def wide_cross_validate(model, X_train, y_train, vocab_size):
  cv_scores = []
  for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = Wide(vocab_size, 1)
    acc = model_train(model, X_train[train], y_train[train], X_train[test], y_train[test])
    print("Accuracy (Wide): %.3f" % acc)
    cv_scores.append(acc)

  # evaluate the model
  acc = np.mean(cv_scores)
  std = np.std(cv_scores)
  # print("Model LogReg accuracy: %.2f%% (+/- %.2f%%)" % (acc2*100, std2*100))
  return print("Model Wide accuracy: %.2f%% (+/- %.2f%%)" % (acc*100, std*100))



In [ ]:
# BOW
#with stopwords
X1_tens_train_full_ws = torch.cat((X1_tens_train_ws, X1_tens_val_ws), 0)
y1_tens_train_full_ws = torch.cat((y1_tens_train_ws, y1_tens_val_ws), 0)

X2_tens_train_full_ws = torch.cat((X2_tens_train_ws, X2_tens_val_ws), 0)
y2_tens_train_full_ws = torch.cat((y2_tens_train_ws, y2_tens_val_ws), 0)

X3_tens_train_full_ws = torch.cat((X3_tens_train_ws, X3_tens_val_ws), 0)
y3_tens_train_full_ws = torch.cat((y3_tens_train_ws, y3_tens_val_ws), 0)

#without lemma
X1_tens_train_full_wol = torch.cat((X1_tens_train_wol, X1_tens_val_wol), 0)
y1_tens_train_full_wol = torch.cat((y1_tens_train_wol, y1_tens_val_wol), 0)

X2_tens_train_full_wol = torch.cat((X2_tens_train_wol, X2_tens_val_wol), 0)
y2_tens_train_full_wol = torch.cat((y2_tens_train_wol, y2_tens_val_wol), 0)

X3_tens_train_full_wol = torch.cat((X3_tens_train_wol, X3_tens_val_wol), 0)
y3_tens_train_full_wol = torch.cat((y3_tens_train_wol, y3_tens_val_wol), 0)

In [ ]:
# TFIDF
# with stopwords
X1_tens_train_tfidf_full_ws = torch.cat((X1_tens_train_tfidf_ws, X1_tens_val_tfidf_ws), 0)
y1_tens_train_tfidf_full_ws = torch.cat((y1_tens_train_tfidf_ws, y1_tens_val_tfidf_ws), 0)

X2_tens_train_tfidf_full_ws = torch.cat((X2_tens_train_tfidf_ws, X2_tens_val_tfidf_ws), 0)
y2_tens_train_tfidf_full_ws = torch.cat((y2_tens_train_tfidf_ws, y2_tens_val_tfidf_ws), 0)

X3_tens_train_tfidf_full_ws = torch.cat((X3_tens_train_tfidf_ws, X3_tens_val_tfidf_ws), 0)
y3_tens_train_tfidf_full_ws = torch.cat((y3_tens_train_tfidf_ws, y3_tens_val_tfidf_ws), 0)


# without lemma
X1_tens_train_tfidf_full_wol = torch.cat((X1_tens_train_tfidf_wol, X1_tens_val_tfidf_wol), 0)
y1_tens_train_tfidf_full_wol = torch.cat((y1_tens_train_tfidf_wol, y1_tens_val_tfidf_wol), 0)

X2_tens_train_tfidf_full_wol = torch.cat((X2_tens_train_tfidf_wol, X2_tens_val_tfidf_wol), 0)
y2_tens_train_tfidf_full_wol = torch.cat((y2_tens_train_tfidf_wol, y2_tens_val_tfidf_wol), 0)

X3_tens_train_tfidf_full_wol = torch.cat((X3_tens_train_tfidf_wol, X3_tens_val_tfidf_wol), 0)
y3_tens_train_tfidf_full_wol = torch.cat((y3_tens_train_tfidf_wol, y3_tens_val_tfidf_wol), 0)

In [ ]:
#BOW CV
print("BOW df1 with stopwords")
model111 = Wide(get_vocab_size(X1_tens_train_full_ws), 1)
df1_bow_lr_eval = wide_cross_validate(model111, X1_tens_train_full_ws, y1_tens_train_full_ws, get_vocab_size(X1_tens_train_full_ws))
print("BOW df2 with stopwords")
model222 = Wide(get_vocab_size(X2_tens_train_full_ws), 1)
df2_bow_lr_eval = wide_cross_validate(model222, X2_tens_train_full_ws, y2_tens_train_full_ws, get_vocab_size(X2_tens_train_full_ws))
print("BOW df3 with stopwords")
model333 = Wide(get_vocab_size(X3_tens_train_full_ws), 1)
df3_bow_lr_eval = wide_cross_validate(model333, X3_tens_train_full_ws, y3_tens_train_full_ws, get_vocab_size(X3_tens_train_full_ws))

print("BOW df1 witout lemma")
model1111 = Wide(get_vocab_size(X1_tens_train_full_wol), 1)
df1_bow_lr_eval = wide_cross_validate(model1111, X1_tens_train_full_wol, y1_tens_train_full_wol, get_vocab_size(X1_tens_train_full_wol))
print("BOW df2 witout lemma")
model2222 = Wide(get_vocab_size(X2_tens_train_full_wol), 1)
df2_bow_lr_eval = wide_cross_validate(model2222, X2_tens_train_full_wol, y2_tens_train_full_wol, get_vocab_size(X2_tens_train_full_wol))
print("BOW df3 witout lemma")
model3333 = Wide(get_vocab_size(X3_tens_train_full_wol), 1)
df3_bow_lr_eval = wide_cross_validate(model3333, X3_tens_train_full_wol, y3_tens_train_full_wol, get_vocab_size(X3_tens_train_full_wol))

BOW df1 with stopwords
Accuracy (Wide): 0.675
Accuracy (Wide): 0.853
Accuracy (Wide): 0.508
Accuracy (Wide): 0.497
Accuracy (Wide): 0.497
Model Wide accuracy: 60.63% (+/- 14.09%)
BOW df2 with stopwords
Accuracy (Wide): 0.669
Accuracy (Wide): 0.740
Accuracy (Wide): 0.746
Accuracy (Wide): 0.721
Accuracy (Wide): 0.729
Model Wide accuracy: 72.10% (+/- 2.74%)
BOW df3 with stopwords
Accuracy (Wide): 0.782
Accuracy (Wide): 0.503
Accuracy (Wide): 0.760
Accuracy (Wide): 0.503
Accuracy (Wide): 0.702
Model Wide accuracy: 64.99% (+/- 12.29%)
BOW df1 witout lemma
Accuracy (Wide): 0.492
Accuracy (Wide): 0.492
Accuracy (Wide): 0.817
Accuracy (Wide): 0.770
Accuracy (Wide): 0.770
Model Wide accuracy: 66.81% (+/- 14.47%)
BOW df2 witout lemma
Accuracy (Wide): 0.668
Accuracy (Wide): 0.760
Accuracy (Wide): 0.687
Accuracy (Wide): 0.663
Accuracy (Wide): 0.726
Model Wide accuracy: 70.09% (+/- 3.69%)
BOW df3 witout lemma
Accuracy (Wide): 0.676
Accuracy (Wide): 0.503
Accuracy (Wide): 0.503
Accuracy (Wide): 0.74

In [ ]:
#TFIDF CV
print("TFIDF df1 with stopwords")
model444 = Wide(get_vocab_size(X1_tens_train_tfidf_full_ws), 1)
df1_TFIDF_lr_eval = wide_cross_validate(model444, X1_tens_train_tfidf_full_ws, y1_tens_train_tfidf_full_ws, get_vocab_size(X1_tens_train_tfidf_full_ws))
print("TFIDF df2 with stopwords")
model555 = Wide(get_vocab_size(X2_tens_train_tfidf_full_ws), 1)
df2_TFIDF_lr_eval = wide_cross_validate(model555, X2_tens_train_tfidf_full_ws, y2_tens_train_tfidf_full_ws, get_vocab_size(X2_tens_train_tfidf_full_ws))
print("TFIDF df3 with stopwords")
model666 = Wide(get_vocab_size(X3_tens_train_tfidf_full_ws), 1)
df3_TFIDF_lr_eval = wide_cross_validate(model666, X3_tens_train_tfidf_full_ws, y3_tens_train_tfidf_full_ws, get_vocab_size(X3_tens_train_tfidf_full_ws))

print("TFIDF df1 witout lemma")
model4444 = Wide(get_vocab_size(X1_tens_train_tfidf_full_wol), 1)
df1_TFIDF_lr_eval = wide_cross_validate(model4444, X1_tens_train_tfidf_full_wol, y1_tens_train_tfidf_full_wol, get_vocab_size(X1_tens_train_tfidf_full_wol))
print("TFIDF df2 witout lemma")
model5555 = Wide(get_vocab_size(X2_tens_train_tfidf_full_wol), 1)
df2_TFIDF_lr_eval = wide_cross_validate(model5555, X2_tens_train_tfidf_full_wol, y2_tens_train_tfidf_full_wol, get_vocab_size(X2_tens_train_tfidf_full_wol))
print("TFIDF df3 witout lemma")
model6666 = Wide(get_vocab_size(X3_tens_train_tfidf_full_wol), 1)
df3_TFIDF_lr_eval = wide_cross_validate(model6666, X3_tens_train_tfidf_full_wol, y3_tens_train_tfidf_full_wol, get_vocab_size(X3_tens_train_tfidf_full_wol))

TFIDF df1 with stopwords
Accuracy (Wide): 0.508
Accuracy (Wide): 0.696
Accuracy (Wide): 0.508
Accuracy (Wide): 0.497
Accuracy (Wide): 0.628
Model Wide accuracy: 56.75% (+/- 8.04%)
TFIDF df2 with stopwords
Accuracy (Wide): 0.700
Accuracy (Wide): 0.500
Accuracy (Wide): 0.720
Accuracy (Wide): 0.634
Accuracy (Wide): 0.621
Model Wide accuracy: 63.50% (+/- 7.71%)
TFIDF df3 with stopwords
Accuracy (Wide): 0.503
Accuracy (Wide): 0.497
Accuracy (Wide): 0.503
Accuracy (Wide): 0.503
Accuracy (Wide): 0.500
Model Wide accuracy: 50.11% (+/- 0.22%)
TFIDF df1 witout lemma
Accuracy (Wide): 0.508
Accuracy (Wide): 0.492
Accuracy (Wide): 0.665
Accuracy (Wide): 0.497
Accuracy (Wide): 0.503
Model Wide accuracy: 53.30% (+/- 6.62%)
TFIDF df2 witout lemma
Accuracy (Wide): 0.506
Accuracy (Wide): 0.637
Accuracy (Wide): 0.636
Accuracy (Wide): 0.500
Accuracy (Wide): 0.717
Model Wide accuracy: 59.91% (+/- 8.38%)
TFIDF df3 witout lemma
Accuracy (Wide): 0.592
Accuracy (Wide): 0.497
Accuracy (Wide): 0.497
Accuracy (Wi

# Similar news

In [ ]:
# create the tf-idf vectorizer
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()
vectorizer3 = TfidfVectorizer()

# fit the vectorizer on the news titles
tfidf_matrix1 = vectorizer1.fit_transform(df1['title'])
tfidf_matrix2 = vectorizer2.fit_transform(df2['title'])
tfidf_matrix3 = vectorizer3.fit_transform(df3['title'])

# calculate cosine distance between all pairs of news titles
cosine_sim1 = 1 - cosine_distances(tfidf_matrix1)
cosine_sim2 = 1 - cosine_distances(tfidf_matrix2)
cosine_sim3 = 1 - cosine_distances(tfidf_matrix3)

In [ ]:
# the index of the first article
index = 0

# get the cosine distances of the first article to all other articles
distances = cosine_sim1[index]

# sort the distances in descending order
sorted_distances = sorted(enumerate(distances), key=lambda x: x[1], reverse=True)

# print the selected article
print('Selected article:\n', df1.iloc[index], '\n')

# print the top 5 most similar articles
for i, distance in sorted_distances[1:6]:
    print(df1.iloc[i])

Selected article:
 title    MEMBERS: EP #5 – DRIVE BY WIRE: ‘Taxi to the U...
fake                                                     1
Name: 22137, dtype: object 

title     What Facebook Just Did About Guns Is Going To...
fake                                                     1
Name: 8442, dtype: object
title    Episode #120 – SUNDAY WIRE: ‘Crisis of Liberty...
fake                                                     1
Name: 22675, dtype: object
title     Matt Damon SHREDS Trump And Wall Street Banke...
fake                                                     1
Name: 6036, dtype: object
title    Episode #203 – SUNDAY WIRE: ‘The Dotard Effect...
fake                                                     1
Name: 21997, dtype: object
title    Boiler Room EP #70 – Sticks, Stones & The Medi...
fake                                                     1
Name: 23284, dtype: object


# Project demonstration
Prepare a summary of your work and discuss your choices with other course mates.

1. Prepare and carry out 5-10 min demo of your results and achievements in this project.
2. Publish your project on the GitHub or GitLab repository
3. Don't remember to add README file with description of this project, specification of your goals and assumptions and explanation of your approach and results.